## Skapa en geojson av SAT leden
* issue [#181](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/181)
* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/181_geojson_SAT_trail)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-09-21 13:11


In [2]:
import requests, json
from lxml import etree
from shapely.geometry import LineString
from shapely.ops import unary_union

OSM_REL = 19012437  # SAT superrelation

def fetch_relation_full(rel_id: int, timeout=60):
    url = f"https://api.openstreetmap.org/api/0.6/relation/{rel_id}/full"
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return etree.fromstring(r.content)

def ways_from_relation_full(xml_root):
    # Bygg upp nod-lexikon
    nodes = {n.attrib["id"]: (float(n.attrib["lon"]), float(n.attrib["lat"])) 
             for n in xml_root.findall("node")}
    # Plocka ut alla ways med deras node-referenser
    ways = {}
    for w in xml_root.findall("way"):
        wid = w.attrib["id"]
        coords = []
        for nd in w.findall("nd"):
            ref = nd.attrib["ref"]
            if ref in nodes:
                coords.append(nodes[ref])
        if len(coords) > 1:
            ways[wid] = coords
    return ways

# 1) Hämta superrelationen
root_super = fetch_relation_full(OSM_REL)

# 2) Hitta alla DIREKTA medlemsrelationer (delrelationer)
member_rel_ids = set()
for rel in root_super.findall("relation"):
    if rel.attrib["id"] == str(OSM_REL):
        for m in rel.findall("member"):
            if m.attrib.get("type") == "relation":
                member_rel_ids.add(int(m.attrib["ref"]))

# 3) Hämta ways från superrelationens egna ways OCH alla delrelationer
all_ways = {}

# ways direkt i superrelationen
super_ways = ways_from_relation_full(root_super)
all_ways.update(super_ways)

# ways i varje delrelation
for rid in sorted(member_rel_ids):
    try:
        rxml = fetch_relation_full(rid)
        sub_ways = ways_from_relation_full(rxml)
        all_ways.update(sub_ways)  # dedup på way-id
    except requests.HTTPError as e:
        print(f"Varn: kunde inte hämta relation {rid}: {e}")

# 4) Gör shapely-linjer
lines = [LineString(coords) for coords in all_ways.values() if len(coords) > 1]
if not lines:
    raise RuntimeError("Hittade inga linjer – avbryter.")
trail = unary_union(lines)
center_lat, center_lon = trail.centroid.y, trail.centroid.x

print(f"Delrelationer: {len(member_rel_ids)} | Ways: {len(all_ways)} | Linjesegment: {len(lines)}")

# 5) (Valfritt) spara som GeoJSON (flattenade LineStrings)
from shapely.geometry import mapping, MultiLineString
from pathlib import Path

features = []
if trail.geom_type == "LineString":
    features = [{"type":"Feature","geometry":mapping(trail),"properties":{}}]
elif trail.geom_type == "MultiLineString":
    for seg in trail.geoms:
        features.append({"type":"Feature","geometry":mapping(seg),"properties":{}})
else:
    # om unary_union råkar ge GeometryCollection
    for seg in getattr(trail, "geoms", []):
        if seg.geom_type in ("LineString","MultiLineString"):
            if seg.geom_type == "LineString":
                features.append({"type":"Feature","geometry":mapping(seg),"properties":{}})
            else:
                for s in seg.geoms:
                    features.append({"type":"Feature","geometry":mapping(s),"properties":{}})

gj = {"type":"FeatureCollection","features":features}
Path("SAT_full.geojson").write_text(json.dumps(gj), encoding="utf-8")
print("Sparade: SAT_full.geojson")


Delrelationer: 19 | Ways: 706 | Linjesegment: 706
Sparade: SAT_full.geojson


In [3]:
    end_time = time.time()
    duration = end_time - start_time
    print(f"Finished in {duration:.2f} seconds.")


Finished in 2.93 seconds.
